# Misc utils

In [ ]:
#| default_exp utils.utils

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()

In [ ]:
# |hide
# Stuff usef for dev/exploration purposes only.
from nbdev.showdoc import *
from fastcore.test import test_eq, is_close

In [ ]:
# |export
# |hide
import sys
from collections import defaultdict
import warnings
import numpy as np
from typing import Optional, Union
from lovely_numpy.utils.config import get_config


In [ ]:
# |hide
# |export

# Do we want this float in decimal or scientific mode?
def sci_mode(f: float):
    config = get_config()
    return ((abs(f) < 10**config.threshold_min) or
            (abs(f) > 10**config.threshold_max))

In [ ]:
# |hide
test_eq(sci_mode(1.), False)
test_eq(sci_mode(0.00001), True)
test_eq(sci_mode(10000000), True)

# It would be fine either way, both `e` and `f` formats handle those.
test_eq(sci_mode(float('nan')), False)
test_eq(sci_mode(float('inf')), True) 

In [ ]:
# |hide

# What's happening in the cell below
fmt = f"{{:.{3}{'e'}}}"
fmt, fmt.format(1.23)

('{:.3e}', '1.230e+00')

In [ ]:
# |export

# Convert an ndarray or scalar into a string.
# This only looks good for small arrays, which is how it's intended to be used.
def pretty_str(x):
    """A slightly better way to print `float`-y values.
    Works for `np.ndarray`, `torch.Tensor`, `jax.DeviceArray`, and scalars."""

    if isinstance(x, int):
        return '{}'.format(x)
    elif isinstance(x, float):
        if x == 0.:
            return "0."

        sci = sci_mode(x) if get_config().sci_mode is None else get_config().sci_mode
        
        fmt = f"{{:.{get_config().precision}{'e' if sci else 'f'}}}"

        return fmt.format(x)
    elif x.ndim == 0:
            return pretty_str(x.item())
    else:
        slices = [pretty_str(x[i]) for i in range(0, x.shape[0])]
        return '[' + ", ".join(slices) + ']'

In [ ]:
# |hide
np.random.seed(0)
randoms = np.random.randn(100)

In [ ]:
nasties = randoms[:12].copy()

nasties[0] *= 10000
nasties[1] /= 10000
nasties[3] = float('inf')
nasties[4] = float('-inf')
nasties[5] = float('nan')
nasties = nasties.reshape((2,6))

In [ ]:
pretty_str(nasties)

'[[1.764e+04, 4.002e-05, 0.979, inf, -inf, nan], [0.950, -0.151, -0.103, 0.411, 0.144, 1.454]]'

In [ ]:
test_eq(pretty_str(nasties), '[[1.764e+04, 4.002e-05, 0.979, inf, -inf, nan], [0.950, -0.151, -0.103, 0.411, 0.144, 1.454]]')

In [ ]:
# |export
def sparse_join(lst, sep=" "):
    # Join non-empty list elements into a space-sepaeated string
    return sep.join( [ l for l in lst if l] )

In [ ]:
# |hide
test_eq(sparse_join(["Hello", None, "World"]), 'Hello World')

In [ ]:
# |export
def ansi_color(s: str, col: str, use_color=True):
        "Very minimal ANSI color support"
        style = defaultdict(str)
        style["grey"] = "\x1b[38;2;127;127;127m"
        style["red"] = "\x1b[31m"
        end_style = "\x1b[0m"
       
        return style[col]+s+end_style if use_color else s

In [ ]:
# |hide
print(ansi_color("Hello, world!", "red") + " Hii!")

Hello, world! Hii!


In [ ]:
# |hide
test_eq(ansi_color("Hello, world", "red") , "\x1b[31mHello, world\x1b[0m")

In [ ]:
# |export
def np_to_str_common(x: Union[np.ndarray, np.generic],  # Input
                        color=True,                     # ANSI color highlighting
                        ddof=0):                        # For "std" unbiasing

    if x.size == 0:
        return ansi_color("empty", "grey", color)

    zeros = ansi_color("all_zeros", "grey", color) if np.equal(x, 0.).all() and x.size > 1 else None
    pinf = ansi_color("+Inf!", "red", color) if np.isposinf(x).any() else None
    ninf = ansi_color("-Inf!", "red", color) if np.isneginf(x).any() else None
    nan = ansi_color("NaN!", "red", color) if np.isnan(x).any() else None

    attention = sparse_join([zeros,pinf,ninf,nan])
    numel = f"n={x.size}" if x.size > 5 and max(x.shape) != x.size else None

    summary = None
    if not zeros and isinstance(x, np.ndarray):
        # Calculate stats on good values only.
        gx = x[ np.isfinite(x) ]

        minmax = f"x∈[{pretty_str(gx.min())}, {pretty_str(gx.max())}]" if gx.size > 2 else None
        meanstd = f"μ={pretty_str(gx.mean())} σ={pretty_str(gx.std(ddof=ddof))}" if gx.size >= 2 else None
        summary = sparse_join([numel, minmax, meanstd])


    return sparse_join([ summary, attention])

In [ ]:
print(np_to_str_common(nasties))

n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!


In [ ]:
# |hide
test_eq(np_to_str_common(nasties), 'n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 \x1b[31m+Inf!\x1b[0m \x1b[31m-Inf!\x1b[0m \x1b[31mNaN!\x1b[0m')

In [ ]:
np_to_str_common(np.array([1., 2, 3]))

'x∈[1.000, 3.000] μ=2.000 σ=0.816'

In [ ]:
# |export
def history_warning():
    "Issue a warning (once) ifw e are running in IPYthon with output cache enabled"
    if "get_ipython" in globals() and get_ipython().cache_size > 0:
        warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")

In [ ]:
# |hide
get_ipython().cache_size=1000
history_warning()
get_ipython().cache_size=0

/tmp/ipykernel_657190/2880814502.py:5: UserWarning: IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html
  warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")


In [ ]:
# |exporti

# functools.cached_property is not available in python < 3.8

assert sys.version_info.major == 3 # Python 4 some day?

if sys.version_info.minor < 8:
    class cached_property:
        attrname: str
        def __init__(self, func):
            self.func = func

        def __set_name__(self, owner, name):
            self.attrname = name

        def __get__(self, instance, owner=None):
            if hasattr(instance, "_cache_" + self.attrname):
                return getattr(instance, "_cache_" + self.attrname)
            else:
                x = self.func(instance)
                setattr(instance, "_cache_" + self.attrname, x)
                return x
else:
    from functools import cached_property


In [ ]:
class Test:
    @cached_property
    def test(self):
        print("property call ")
        return 123

    def __repr__(self):
        return "Test object"

In [ ]:
t=Test()
print(t.test)
print(t.test)
t

property call 
123
123


Test object

In [ ]:
# |export 
def in_debugger():
    """Returns True if a debugger was used.
    
    Note: This funciton will keep returning True even after you exit the debugger."""
    return getattr(sys, "gettrace", None) and sys.gettrace() is not None

In [ ]:
test_eq(in_debugger(), False)